_Main topics covered during today's session:_

Previous NB:

1. **Notes on Pandas, Part 0 (from Topic 7), introducing Series and DataFrame objects**

This NB:

2. **Notes on Pandas, Part 0 (from Topic 7), Series and DataFrame manipulations**

We are simply splitting up this NB that is on Vocareum and adding some explanatory notes for this Skills OH session. The code content is the mostly the same. We note below where there are differences.

**The below two cells simply set up the NB with the same two df's as the previous NB (cafes and cafes2).**

In [8]:
import pandas as pd  # Standard idiom for loading pandas
from pandas import DataFrame, Series

In [9]:
cafes = DataFrame({'name': ['east pole', 'chrome yellow', 'brash', 'taproom', '3heart', 'spiller park pcm', 'refuge', 'toptime'],
                   'zip': [30324, 30312, 30318, 30317, 30306, 30308, 30303, 30318],
                   'poc': ['jared', 'kelly', 'matt', 'jonathan', 'nhan', 'dale', 'kitti', 'nolan']})

cafes2 = cafes[['poc', 'zip']]
cafes2.index = cafes['name']
cafes2.index.name = None
cafes2

,poc,zip
east pole,jared,30324
chrome yellow,kelly,30312
brash,matt,30318
taproom,jonathan,30317
3heart,nhan,30306
spiller park pcm,dale,30308
refuge,kitti,30303
toptime,nolan,30318


**Now let's continue, working on manipulating dataframes (and Series).**

Adding columns is easy. Suppose every cafe has a 4-star rating on Yelp! and a two-dollar-sign cost:

In [10]:
cafes2['rating'] = 4.0
cafes2['price'] = '$$'
cafes2

,poc,zip,rating,price
east pole,jared,30324,4.0,$$
chrome yellow,kelly,30312,4.0,$$
brash,matt,30318,4.0,$$
taproom,jonathan,30317,4.0,$$
3heart,nhan,30306,4.0,$$
spiller park pcm,dale,30308,4.0,$$
refuge,kitti,30303,4.0,$$
toptime,nolan,30318,4.0,$$


In [11]:
cafes2.iloc[[1, 3]]

,poc,zip,rating,price
chrome yellow,kelly,30312,4.0,$$
taproom,jonathan,30317,4.0,$$


And vector arithmetic should work on columns as expected.

In [12]:
prices_as_ints = cafes2['price'].apply(lambda s: len(s))
prices_as_ints

east pole           2
chrome yellow       2
brash               2
taproom             2
3heart              2
spiller park pcm    2
refuge              2
toptime             2
Name: price, dtype: int64

What did we do here?

1. Created a new Series, called prices_as_ints.
2. We used a lambda function (remember those?) and .apply() to populate the Series.
3. In the lambda function, the length of s is always 2 ($$ in each row)

The ``apply()`` method lets you apply an arbitrary function to the group results.
The method takes a ``DataFrame``, and returns either a Pandas object (e.g., ``DataFrame``, ``Series``) or a scalar; the combine operation will be tailored to the type of output returned.

Each value of a series (dataframe column) is passed through the function inside .apply()

There are 3 types of functions you can pass in apply : 
1. Built-in functions
2. User-defined functions
3. Lambda functions

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

Let's look at some examples.

In [21]:
# create a df to work with
df = pd.DataFrame(
                    {
                        'Name': ['Edward','Natalie','Chris M','Priyatham'],
                        'Sex':  ['M','F','M','M'],
                        'Age':  [45,35,29,26],
                        'weight(kgs)': [68.4,58.2,64.3,53.1]
                    }
                )
df

,Name,Sex,Age,weight(kgs)
0,Edward,M,45,68.4
1,Natalie,F,35,58.2
2,Chris M,M,29,64.3
3,Priyatham,M,26,53.1


In [22]:
# using a built-in function
df['Name'].apply(len)

0    6
1    7
2    7
3    9
Name: Name, dtype: int64

In [23]:
# using a user-defined function
def add_age(age):
    return age+1

df['Age'].apply(add_age)

0    46
1    36
2    30
3    27
Name: Age, dtype: int64

In [24]:
# now use .apply() to change the value in the df
print(df)
df['Age'] = df['Age'].apply(add_age)
df

        Name Sex  Age  weight(kgs)
0     Edward   M   45         68.4
1    Natalie   F   35         58.2
2    Chris M   M   29         64.3
3  Priyatham   M   26         53.1


,Name,Sex,Age,weight(kgs)
0,Edward,M,46,68.4
1,Natalie,F,36,58.2
2,Chris M,M,30,64.3
3,Priyatham,M,27,53.1


In [25]:
# now use a lambda function to do the same as the add_age function
df['Age'].apply(lambda x: x+1)

0    47
1    37
2    31
3    28
Name: Age, dtype: int64

In [26]:
# compare the two approaches
df['Age'].apply(lambda x: x+1) == df['Age'].apply(add_age)

0    True
1    True
2    True
3    True
Name: Age, dtype: bool

**Note the axis parameter in the .apply() syntax. The default is axis=0, meaning that the function will be applied on a column basis. However, you can also use axis=1, which means that the function will be applied to the rows of the dataframe. See the slide for a visual of how this is done.**

Now let's go back to our cafes data. Recall that we created the prices_as_ints series.

In [27]:
# prices_as_ints = cafes2['price'].apply(lambda s: len(s))
prices_as_ints

east pole           2
chrome yellow       2
brash               2
taproom             2
3heart              2
spiller park pcm    2
refuge              2
toptime             2
Name: price, dtype: int64

Let's create a new column.

In [28]:
cafes2['value'] = cafes2['rating'] / prices_as_ints
cafes2

,poc,zip,rating,price,value
east pole,jared,30324,4.0,$$,2.0
chrome yellow,kelly,30312,4.0,$$,2.0
brash,matt,30318,4.0,$$,2.0
taproom,jonathan,30317,4.0,$$,2.0
3heart,nhan,30306,4.0,$$,2.0
spiller park pcm,dale,30308,4.0,$$,2.0
refuge,kitti,30303,4.0,$$,2.0
toptime,nolan,30318,4.0,$$,2.0


Because the columns are `Series` objects, there is an implicit matching that is happening on the indexes. In the preceding example, it works because all the `Series` objects involved have identical indexes.

However, the following will not work as intended because referencing rows yields copies.

For instance, suppose there is a price hike of one more `'$'` for being in the 30306 and 30308 zip codes. (If you are in Atlanta, you may know that these are the zip codes that place you close to, or in, [Ponce City Market](http://poncecitymarket.com/) and the [Eastside Beltline Trail](https://beltline.org/explore-atlanta-beltline-trails/eastside-trail/)!) Let's increase the price there, on a copy of the dataframe, `cafes3`.

In [58]:
cafes3 = cafes2.copy()
cafes3

,poc,zip,rating,price,value
east pole,jared,30324,4.0,$$,2.0
chrome yellow,kelly,30312,4.0,$$,2.0
brash,matt,30318,4.0,$$,2.0
taproom,jonathan,30317,4.0,$$,2.0
3heart,nhan,30306,4.0,$$,2.0
spiller park pcm,dale,30308,4.0,$$,2.0
refuge,kitti,30303,4.0,$$,2.0
toptime,nolan,30318,4.0,$$,2.0


In [59]:
# create a boolean mask
is_fancy = cafes3['zip'].isin({30306, 30308})
# Alternative:
#is_fancy = cafes3['zip'].apply(lambda z: z in {30306, 30308})
is_fancy # this is the mask

east pole           False
chrome yellow       False
brash               False
taproom             False
3heart               True
spiller park pcm     True
refuge              False
toptime             False
Name: zip, dtype: bool

In [60]:
# apply the boolean mask to only return the True rows
cafes3[is_fancy]

,poc,zip,rating,price,value
3heart,nhan,30306,4.0,$$,2.0
spiller park pcm,dale,30308,4.0,$$,2.0


In [61]:
# Recall: Legal Python for string concatenation
s = '$$'
s += '$'
print(s)

$$$


In [62]:
cafes3[is_fancy]['price'] += '$'

C:\Users\lrhsh\AppData\Local\Temp\ipykernel_32992\3738753830.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cafes3[is_fancy]['price'] += '$'


What does that error message mean? Let's see if anything changed.

In [63]:
cafes3

,poc,zip,rating,price,value
east pole,jared,30324,4.0,$$,2.0
chrome yellow,kelly,30312,4.0,$$,2.0
brash,matt,30318,4.0,$$,2.0
taproom,jonathan,30317,4.0,$$,2.0
3heart,nhan,30306,4.0,$$,2.0
spiller park pcm,dale,30308,4.0,$$,2.0
refuge,kitti,30303,4.0,$$,2.0
toptime,nolan,30318,4.0,$$,2.0


Nope! When you slice horizontally, you get copies of the original data, not references to subsets of the original data(see the error message text). Therefore, we'll need different strategy.

Observe that the error message suggests a way!

In [64]:
cafes3.loc[is_fancy, 'price'] += '$'
cafes3

,poc,zip,rating,price,value
east pole,jared,30324,4.0,$$,2.0
chrome yellow,kelly,30312,4.0,$$,2.0
brash,matt,30318,4.0,$$,2.0
taproom,jonathan,30317,4.0,$$,2.0
3heart,nhan,30306,4.0,$$$,2.0
spiller park pcm,dale,30308,4.0,$$$,2.0
refuge,kitti,30303,4.0,$$,2.0
toptime,nolan,30318,4.0,$$,2.0


**In this NB on Vocareum, at this point it discusses a different manner for achieving the same result. We are eliminating that from our discussion here, and we have remove those cells. We encourage you to take a look at the code, for your own understanding.**

We will simply make the cafes4 dataframe as a copy of the cafes3, so that our code below aligns with the original nb.

In [66]:
cafes4 = cafes3.copy()

**More on `apply()` for `DataFrame` objects.** As with a `Series`, there is a `DataFrame.apply()` procedure. However, it's meaning is a bit more nuanced because a `DataFrame` is generally 2-D rather than 1-D.

https://www.geeksforgeeks.org/python-repr-function/

In [67]:
# the default for .apply() is axis=0, recall the slide visual
print(cafes4)
# axis=0 gives the columns
cafes4.apply(lambda x: repr(type(x))) # What does this do? What does the output tell you?

                       poc    zip  rating price  value
east pole            jared  30324     4.0    $$    2.0
chrome yellow        kelly  30312     4.0    $$    2.0
brash                 matt  30318     4.0    $$    2.0
taproom           jonathan  30317     4.0    $$    2.0
3heart                nhan  30306     4.0   $$$    2.0
spiller park pcm      dale  30308     4.0   $$$    2.0
refuge               kitti  30303     4.0    $$    2.0
toptime              nolan  30318     4.0    $$    2.0


poc       <class 'pandas.core.series.Series'>
zip       <class 'pandas.core.series.Series'>
rating    <class 'pandas.core.series.Series'>
price     <class 'pandas.core.series.Series'>
value     <class 'pandas.core.series.Series'>
dtype: object

A useful parameter is `axis`:

In [68]:
# axis=1 gives the rows
cafes4.apply(lambda x: repr(type(x)), axis=1) # What does this do? What does the output tell you?

east pole           <class 'pandas.core.series.Series'>
chrome yellow       <class 'pandas.core.series.Series'>
brash               <class 'pandas.core.series.Series'>
taproom             <class 'pandas.core.series.Series'>
3heart              <class 'pandas.core.series.Series'>
spiller park pcm    <class 'pandas.core.series.Series'>
refuge              <class 'pandas.core.series.Series'>
toptime             <class 'pandas.core.series.Series'>
dtype: object

In [69]:
# errors out
# cafes4.apply(lambda x: (type(x)))

And just to quickly verify what you get when `axis=1`:

In [70]:
# this is a complex lambda function
cafes4.apply(lambda x: print('==> ' + x.name + '\n' + repr(x)) if x.name == 'east pole' else None, axis=1);

==> east pole
poc       jared
zip       30324
rating      4.0
price        $$
value       2.0
Name: east pole, dtype: object


So what does the above lambda function do?

1. It is a lambda function, so it is looking at every row, with the parameter axis=1
2. It checks if the row's name is 'east pole'
3. If the name is 'east pole', it does the print
4. If the name is not 'east pole', it returns None (does not print anything)

**Exercise.** Use `DataFrame.apply()` to update the `'value'` column in `cafes4`, which is out of date given the update of the prices.

In [71]:
cafes4 # Verify visually that `'value'` is out of date

,poc,zip,rating,price,value
east pole,jared,30324,4.0,$$,2.0
chrome yellow,kelly,30312,4.0,$$,2.0
brash,matt,30318,4.0,$$,2.0
taproom,jonathan,30317,4.0,$$,2.0
3heart,nhan,30306,4.0,$$$,2.0
spiller park pcm,dale,30308,4.0,$$$,2.0
refuge,kitti,30303,4.0,$$,2.0
toptime,nolan,30318,4.0,$$,2.0


In [72]:
# .apply() with a user-defined function
def calc_value(row):
    return row['rating'] / len(row['price'])

cafes4['value'] = cafes4.apply(calc_value, axis=1)
cafes4

,poc,zip,rating,price,value
east pole,jared,30324,4.0,$$,2.000000
chrome yellow,kelly,30312,4.0,$$,2.000000
brash,matt,30318,4.0,$$,2.000000
taproom,jonathan,30317,4.0,$$,2.000000
3heart,nhan,30306,4.0,$$$,1.333333
spiller park pcm,dale,30308,4.0,$$$,1.333333
refuge,kitti,30303,4.0,$$,2.000000
toptime,nolan,30318,4.0,$$,2.000000


#### This concludes our discussion for today. We will conclude our Pandas introduction in the next Skills OH session.